Bu repo görsel üreten bir gan modelini ele alır (vanilla)
Bu repo üzerinde 2 aşamada conditional tabular data için veri üretiyor olacağız
* Koşullu yapma
* Tabular datada çalışıyor hale getirme

Şu anda 1. aşamada çalışılınıyor

In [ ]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path 

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

#### Prepare MNIST data

In [ ]:
data_path = Path("./gan/datasets/data")

In [ ]:
from keras.datasets import mnist

In [ ]:
(train_image, train_labels), (_, _) = mnist.load_data()

In [ ]:
train_images = train_image.reshape(train_image.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 32
latent_dim = 100

In [ ]:
# belirli bir koşul için veri seti hazırlama (koşul "= 5" olması)
indices = np.where(train_labels == 5)[0]
train_images_5 = []
for i in range(len(train_labels)):
    if i in indices:
        train_images_5.append(train_images[i])
        
train_images_5 = np.array(train_images_5)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images_5).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# toplamda 60.000 görsel var
train_images.shape

In [ ]:
# toplamda 5421 adet "5" görseli var
train_images_5.shape

Aşağıdaki model oluşturma adımında hangi koşul yazılırsa yazılsın model gerçek veya sahte ayrımı yapmaya çalışacaktır.
Bu yüzden önemli nokta veri setini ayrıştırıp ona vermektir örneğin sadece 5 için görsel çizdirmeye çalışalım.

#### Prepare Model

In [ ]:
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [ ]:
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)

In [ ]:
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)

In [ ]:
gan.fit(train_dataset, epochs=10)

#### Testing Generator

In [ ]:
noise = tf.random.normal([1, 100])

In [ ]:
generated_image = generator(noise)

In [ ]:
generator.summary()

In [ ]:
pred = discriminator(generated_image)

In [ ]:
pred

In [ ]:
plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
# deney
# ilk deney koşula uygun n adet yani birden fazla sentetik veri üretebilir miyiz?
# ikinci deney farklı koşullar için tekrar ve tekrar model eğitmek ile tek bir model eğitmek arasında ne gibi maliyet farkı var?

In [ ]:
# deney 1:

In [ ]:
latent_dim = 100
num_images = 49
random_latent_vectors = tf.random.normal(shape=(num_images, latent_dim))
generated_images = generator(random_latent_vectors)

In [ ]:
plt.imshow(generated_images[10])

In [ ]:
# deney 2:

In [ ]:
%%time
# vanilla gan tüm veriler ile eğitiliyor.
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(train_dataset, epochs=50)

32 dk 46 saniye - 60000x28x28

In [ ]:
%%time
for spec in range(10):
    indices = np.where(train_labels == spec)[0]
    train_images_spec = []
    for i in range(len(train_labels)):
        if i in indices:
            train_images_spec.append(train_images[i])

    train_images_spec = np.array(train_images_spec)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images_spec).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    print("for spec:", spec)
    print("len train_images_spec:", len(train_images_spec))
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(train_dataset, epochs=50)

33 dk - her veri ayrı ayrı

In [ ]:
# Aşarısı sadece modellerin çıktısını test etmek için

In [ ]:
%%time
list_img = []
for spec in range(10):
    indices = np.where(train_labels == spec)[0]
    train_images_spec = []
    for i in range(len(train_labels)):
        if i in indices:
            train_images_spec.append(train_images[i])

    train_images_spec = np.array(train_images_spec)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images_spec).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    print("spec:", spec)
    print("len train_images_spec:", len(train_images_spec))
    print("ilk on:", indices[:10])
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(train_dataset, epochs=20)
    noise = tf.random.normal([1, 100])
    generated_image = generator(noise)
    list_img.append(generated_image[0, :, :, 0])
    #plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
for i in range(10):
    plt.imshow(list_img[i])
    plt.show()

Not: Şu anda ilk aşama tamamlandı. Yani görsel veriler için belirli bir koşula uygun veri üretimi vanilla gan kullanılarak gerçekleştirildi.

Birden fazla örnek oluşturma veya oluşturulan örneklerin gösterilmesi ile ilgili bir problem gözlemlenmedi.

Ayrıca modellerin tek tek eğitilmesi veya tüm verilerin aynı anda koşuldan bağımsız eğitilmesi konusunda bir zaman farkı bulunmamaktadır.

Bizim verilerimizin çok fazla boyuta sahip olduğu için bu kadar uzun sürdü eğitimler örneğin, adult veri seti 32.500x15 boyutundadır, bizim görsel veri setimiz 60.000x48x48 yani yaklaşık 96 katı boyutunda.

Burada bir sonraki aşama tabular data için eğitim gerçekleştirebilmektir.
Daha sonrasında işlemlerin fonksiyonlaştırılması ve nesneye yönelimli programlama yapısında py dosyası formatına getirilmesidir.

En sonunda da bu modelin eğitimi hızlandırılmaya çalışılabilir. Veriyi temsil eden en iyi bir örneklem seçilebilir vs. veya paralelleştirme denenebilir. Modelin içine koşul verilmeye çalışılabilir. Early stopping eklenebilir. Veri boyutuna bağlı epoch sayısı belirlenebilir. Fakat version 1 için elde çalışan bir yapının bulunması iyi olacaktır. 

Şu anda 2. aşamadayız yani görsel verisi için eğittiğimiz modeli tabular data için eğiteceğiz.

In [1]:
from __future__ import absolute_import, division

import tensorflow as tf
import tensorflow.keras as keras 

import numpy as np
from pathlib import Path 

import PIL 
import imageio
from IPython import display

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
import matplotlib.pyplot as plt
import sys 

sys.path.insert(0, "..")

from gan.networks import Generator, Discriminator, GAN

2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [3]:
data_path = "C:/Users/kalybeai-dxlc693/Desktop/GANS/adult.csv"

In [4]:
data = pd.read_csv(data_path)

In [5]:
df = data.copy()

In [6]:
le = preprocessing.LabelEncoder()
for i in ['workclass','education','marital.status','occupation','relationship','race','sex','native.country','income']:
    df[i] = le.fit_transform(df[i].astype(str))

In [7]:
df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,0,77053,11,9,6,0,1,4,0,0,4356,40,39,0
1,82,4,132870,11,9,6,4,1,4,0,0,4356,18,39,0
2,66,0,186061,15,10,6,0,4,2,0,0,4356,40,39,0
3,54,4,140359,5,4,0,7,4,4,0,0,3900,40,39,0
4,41,4,264663,15,10,5,10,3,4,0,0,3900,40,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,4,310152,15,10,4,11,1,4,1,0,0,40,39,0
32557,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32558,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32559,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0


In [35]:
scaler = StandardScaler()

X_train = scaler.fit_transform(df.drop('income', 1))
y_train = df['income'].values

C:\Users\KALYBE~1\AppData\Local\Temp/ipykernel_13708/2975329542.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = scaler.fit_transform(df.drop('income', 1))


In [9]:
X_train.shape

(32561, 14)

In [21]:
y_train.shape

(32561,)

In [23]:
y_train

array([0, 0, 0, ..., 1, 0, 0])

In [10]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 32
latent_dim = 100

In [11]:
def loss_fn(labels, output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(labels, output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.05)

In [12]:
X_train = np.float32(X_train)
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [13]:
%%time
# vanilla gan tüm veriler ile eğitiliyor.
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)

Wall time: 29 ms


In [14]:
gan.fit(train_dataset, epochs=50)

Epoch 1/50
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)


C:\Users\kalybeai-dxlc693\anaconda3\envs\gpu_tensorflow\lib\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
1018/1018 [==============================] - 9s 7ms/step - d_loss: 0.6039 - g_loss: 0.9397
Epoch 2/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.5119 - g_loss: 1.1676
Epoch 3/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.4118 - g_loss: 1.4352
Epoch 4/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3083 - g_loss: 1.8182
Epoch 5/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3074 - g_loss: 1.8499
Epoch 6/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2884 - g_loss: 1.9946
Epoch 7/50
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.2690 - g_loss: 2.1346
Epoch 8/50
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.2584 - g_loss: 2.

In [15]:
noise = tf.random.normal([1, 100])

In [17]:
generated_data = generator(noise)

In [19]:
np.array(generated_data)[0]

array([-0.9999978 ,  0.09759989, -0.41890234,  0.16888426, -0.42873618,
        1.        ,  0.5243812 ,  0.9686602 ,  0.37340143,  0.7217367 ,
       -0.15365264, -0.23727411, -0.00897848,  0.296518  ], dtype=float32)

In [ ]:
# önce train içindeki kısım incelenecek
# sonra koşula uygun yapılmaya çalışılacaktır
# class sayısı dinamik yapılacak

In [69]:
data.columns[:]

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [70]:
generated_data = pd.DataFrame(columns=data.columns[:])

In [71]:
generated_data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income


In [83]:
generated_df = pd.DataFrame(columns=data.columns[:])
X_train = np.float32(X_train)
conditional_datasets = []
list_condition = [0, 1]
for cond in list_condition:
    indices = np.where(y_train == cond)[0]
    train_data_cond = []
    for i in range(len(y_train)):
        if i in indices:
            train_data_cond.append(X_train[i])
    train_data_cond = np.array(train_data_cond)
    train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    conditional_datasets.append(train_dataset)
    
    discriminator = Discriminator()
    generator = Generator()
    gan = GAN(discriminator, generator, latent_dim)
    gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
    gan.fit(conditional_datasets[cond], epochs=50)
    
    num_gen = len(indices)
    random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
    generated_data = generator(random_latent_vectors)
    gen = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
    gen[data.columns[-1]] = cond
    generated_df = pd.concat([generated_df, gen], ignore_index=True)
    

Epoch 1/50
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
1018/1018 [==============================] - 8s 7ms/step - d_loss: 0.5400 - g_loss: 1.0819
Epoch 2/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3344 - g_loss: 1.7635
Epoch 3/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3055 - g_loss: 1.7802
Epoch 4/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2926 - g_loss: 1.8636
Epoch 5/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2738 - g_loss: 2.0141
Epoch 6/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2648 - g_l

1018/1018 [==============================] - 8s 7ms/step - d_loss: 0.1784 - g_loss: 2.8157
Epoch 23/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1750 - g_loss: 2.8486
Epoch 24/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1720 - g_loss: 2.9235
Epoch 25/50
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.1727 - g_loss: 2.8965
Epoch 26/50
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.1720 - g_loss: 2.9103
Epoch 27/50
1018/1018 [==============================] - 7s 6ms/step - d_loss: 0.1695 - g_loss: 2.9640
Epoch 28/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1667 - g_loss: 2.9906
Epoch 29/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1603 - g_loss: 3.0897
Epoch 30/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1601 - g_loss: 3.0896
Epoch 31/50
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.1557

In [84]:
generated_df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,1.000000,-0.948769,-1.000000,1.000000,-0.754838,0.999666,-1.000000,1.000000,-0.999583,-1.000000,-0.101545,0.129699,0.999996,0.060339,0
1,1.000000,-0.383098,-1.000000,0.991013,-0.066472,-0.999389,-1.000000,1.000000,-0.999321,-0.999996,-0.116576,-0.012319,0.992619,0.190698,0
2,1.000000,-0.654344,1.000000,0.998898,0.979635,1.000000,-0.996139,0.999994,-0.999482,-1.000000,-0.162446,0.148838,-0.999995,0.334584,0
3,1.000000,0.339175,-1.000000,0.999990,-0.037939,-0.999998,0.999999,0.999949,-0.991710,-0.999851,-0.170849,-0.077039,1.000000,0.259705,0
4,1.000000,0.010208,-1.000000,0.999306,0.996548,1.000000,0.999999,1.000000,-0.998225,-0.999989,-0.125909,0.343030,-0.996460,0.235007,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.055512,0.097827,-0.013413,0.175842,-0.403104,1.000000,-0.048213,-0.259645,0.403551,0.693283,-0.144101,-0.211437,0.021732,0.300367,1
32557,0.547526,0.089584,0.317269,0.179078,-0.403339,-0.409997,0.995392,-0.912349,0.394151,0.687365,-0.145209,-0.207981,0.038356,0.284578,1
32558,-0.997682,0.087117,0.318418,0.173505,-0.398885,-0.409025,0.011728,-0.910195,0.387590,0.689025,-0.142008,-0.210578,-0.024318,0.291520,1
32559,-0.891580,0.106836,-0.990976,0.187298,-0.412992,-0.411350,-0.858522,-0.894495,0.392411,0.688943,-0.143461,-0.203109,-0.012399,0.289575,1


In [40]:
conditional_datasets[0]
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(conditional_datasets[0], epochs=10)
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = cond

Epoch 1/10
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
1018/1018 [==============================] - 8s 7ms/step - d_loss: 0.5307 - g_loss: 1.1262
Epoch 2/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3802 - g_loss: 1.5279
Epoch 3/10
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.2954 - g_loss: 1.8628
Epoch 4/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2894 - g_loss: 1.9202
Epoch 5/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2744 - g_loss: 2.0435
Epoch 6/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2590 - g_l

In [39]:
conditional_datasets[0]
discriminator = Discriminator()
generator = Generator()
gan = GAN(discriminator, generator, latent_dim)
gan.compile(discriminator_optimizer, generator_optimizer, loss_fn)
gan.fit(conditional_datasets[1], epochs=10)
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen1 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen1[data.columns[-1]] = cond

Epoch 1/10
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
data: Tensor("data:0", shape=(None, 14), dtype=float32)
batch_size: Tensor("strided_slice:0", shape=(), dtype=int32)
random_latent_vectors: Tensor("random_normal:0", shape=(None, 100), dtype=float32)
1018/1018 [==============================] - 8s 7ms/step - d_loss: 0.5404 - g_loss: 1.0693
Epoch 2/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3617 - g_loss: 1.6112
Epoch 3/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.3121 - g_loss: 1.7945
Epoch 4/10
1018/1018 [==============================] - 6s 6ms/step - d_loss: 0.3034 - g_loss: 1.8676
Epoch 5/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2882 - g_loss: 1.9626
Epoch 6/10
1018/1018 [==============================] - 7s 7ms/step - d_loss: 0.2727 - g_l

In [43]:
num_gen = list_count[cond]
random_latent_vectors = tf.random.normal(shape=(num_gen, latent_dim))
generated_data = generator(random_latent_vectors)
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = cond

In [62]:
gen0 = pd.DataFrame(np.array(generated_data), columns=data.columns[:-1])
gen0[data.columns[-1]] = 0

In [63]:
data.columns[-1]

'income'

In [59]:
data.columns[:-1]

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country'],
      dtype='object')